In [2]:
import tensorflow as tf
print(tf.__version__)

# Load the diabetes dataset

from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

# Save the input and target variables

from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']
# Split the data set into training and test sets

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])
# Compile the model

model.compile(loss='mse',
                optimizer="adam",metrics=["mse","mae"])



2.17.0


D:\Anaconda1\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Define the learning rate schedule function

def lr_function(epoch, lr):
    if epoch % 2 == 0:
        return lr
    else:
        return lr + epoch/1000

# Train the model

history = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_function, verbose=1)], verbose=False)

# Train the model with a difference schedule

history = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(lambda x:1/(3+5*x), verbose=1)], 
                    verbose=False)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.

Epoch 2: LearningRateScheduler setting learning rate to 0.0020000000474974513.

Epoch 3: LearningRateScheduler setting learning rate to 0.0020000000949949026.

Epoch 4: LearningRateScheduler setting learning rate to 0.005000000094994903.

Epoch 5: LearningRateScheduler setting learning rate to 0.004999999888241291.

Epoch 6: LearningRateScheduler setting learning rate to 0.009999999888241292.

Epoch 7: LearningRateScheduler setting learning rate to 0.009999999776482582.

Epoch 8: LearningRateScheduler setting learning rate to 0.01699999977648258.

Epoch 9: LearningRateScheduler setting learning rate to 0.016999999061226845.

Epoch 10: LearningRateScheduler setting learning rate to 0.025999999061226846.

Epoch 1: LearningRateScheduler setting learning rate to 0.3333333333333333.

Epoch 2: LearningRateScheduler setting learning rate to 0.125.

Epoch 3: LearningRateScheduler setting learning rate to 0.0769230

In [6]:
# Train the model with a CSV logger

history = model.fit(train_data, train_targets, epochs=10,
                    callbacks=[tf.keras.callbacks.CSVLogger("results.csv")], verbose=False)
# Load the CSV

import pandas as pd

pd.read_csv("results.csv", index_col='epoch')

,loss,mae,mse,val_loss,val_mae,val_mse
epoch,,,,,,
0,27289.589844,146.488708,27289.589844,NaN,NaN,NaN
1,27214.033203,146.224823,27214.033203,NaN,NaN,NaN
2,27136.695312,145.958191,27136.695312,NaN,NaN,NaN
3,27059.490234,145.696579,27059.490234,NaN,NaN,NaN
4,26984.234375,145.434555,26984.234375,NaN,NaN,NaN
5,26907.505859,145.172577,26907.505859,NaN,NaN,NaN
6,26831.583984,144.910446,26831.583984,NaN,NaN,NaN
7,26755.503906,144.648544,26755.503906,NaN,NaN,NaN
8,26680.083984,144.388031,26680.083984,NaN,NaN,NaN


In [8]:
# Print the epoch number at the beginning of each epoch

epoch_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_begin=lambda epoch,logs: print('Starting Epoch {}!'.format(epoch+1)))
# Print the loss at the end of each batch

batch_loss_callback = tf.keras.callbacks.LambdaCallback(
    on_batch_end=lambda batch,logs: print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss'])))
# Inform that training is finished

train_finish_callback = tf.keras.callbacks.LambdaCallback(
    on_train_end=lambda logs: print('Training finished!'))

# Train the model with the lambda callbacks

history = model.fit(train_data, train_targets, epochs=5, batch_size=100,
                    callbacks=[epoch_callback, batch_loss_callback,train_finish_callback], verbose=False)

Starting Epoch 1!

 After batch 0, the loss is 30757.92.

 After batch 1, the loss is 26227.70.

 After batch 2, the loss is 27359.75.

 After batch 3, the loss is 26553.49.
Starting Epoch 2!

 After batch 0, the loss is 25851.21.

 After batch 1, the loss is 27887.88.

 After batch 2, the loss is 27193.30.

 After batch 3, the loss is 26530.51.
Starting Epoch 3!

 After batch 0, the loss is 24327.82.

 After batch 1, the loss is 23843.34.

 After batch 2, the loss is 26407.32.

 After batch 3, the loss is 26507.19.
Starting Epoch 4!

 After batch 0, the loss is 24832.03.

 After batch 1, the loss is 24061.41.

 After batch 2, the loss is 25404.41.

 After batch 3, the loss is 26484.14.
Starting Epoch 5!

 After batch 0, the loss is 28660.60.

 After batch 1, the loss is 26717.54.

 After batch 2, the loss is 26426.69.

 After batch 3, the loss is 26461.49.
Training finished!


In [10]:
# Train the model with the ReduceLROnPlateau callback

history = model.fit(train_data, train_targets, epochs=100, batch_size=100,
                    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="loss",factor=0.2, verbose=1)], verbose=False)